# CRDS reference mappings

This notebook explores the CRDS **.rmap** rules files.

## Prerequisites

To follow along with the examples in this notebook, you will need:

- The following Python packages installed:

```
$ pip install crds==7.6.1
```

In [ ]:
import crds

## .rmap olympics

Congratulations, you've been chosen to represent your scrum team in the .rmap olympics!  For each of the following challenges, edit the `RMAP_CONTENT` variable according to the instructions.  Test your changes by executing the "Test your solution" code cell.

### Challenge 1

To celebrate your humble notebook author's birthday, we're going to change the NIRCam `photom` reference file to a special edition just for that day.  Modify the following .rmap to select `jwst_nircam_photom_birthday.fits` for the duration of 11/20/2020, but only for the `NRC_IMAGE` exposure type.

In [ ]:
RMAP_CONTENT = """
header = {
    'classes' : ('Match', 'UseAfter'),
    'derived_from' : 'jwst_nircam_photom_0009.rmap',
    'filekind' : 'PHOTOM',
    'instrument' : 'NIRCAM',
    'mapping' : 'REFERENCE',
    'name' : 'jwst_nircam_photom_0010.rmap',
    'observatory' : 'JWST',
    'parkey' : (('META.INSTRUMENT.DETECTOR', 'META.EXPOSURE.TYPE'), ('META.OBSERVATION.DATE', 'META.OBSERVATION.TIME')),
    'sha1sum' : '32715a282c1417c74b06eaac369d1c1bb99ba248',
}

selector = Match({
    ('NRCA1', 'N/A') : UseAfter({
        '1900-01-01 00:00:00' : 'jwst_nircam_photom_0031.fits',
        '2014-01-01 00:00:00' : 'jwst_nircam_photom_0048.fits',
    }),
    ('NRCA1', 'NRC_CORON|NRC_FLAT|NRC_FOCUS|NRC_IMAGE|NRC_TACONFIRM|NRC_TACQ|NRC_TSIMAGE') : UseAfter({
        '2014-01-01 00:00:00' : 'jwst_nircam_photom_0074.fits',
    }),
})
"""

#### Test your solution

In [ ]:
rmap = crds.core.rmap.ReferenceMapping.from_string(RMAP_CONTENT, ignore_checksum=True)

def assert_best_ref(exposure_type, date, filename):
    result = rmap.get_best_ref(
        {
            "META.INSTRUMENT.DETECTOR": "NRCA1",
            "META.EXPOSURE.TYPE": exposure_type,
            "META.OBSERVATION.DATE": date,
            "META.OBSERVATION.TIME": "00:00:00",
        }
    )
    
    if result != filename:
        message = f"Test failed for META.EXPOSURE.TYPE={exposure_type}, META.OBSERVATION.DATE={date}.  Expected {filename}, got {result}"
        raise AssertionError(message)
    
assert_best_ref("SOME EXPOSURE TYPE", "1980-01-01", "jwst_nircam_photom_0031.fits")
assert_best_ref("SOME EXPOSURE TYPE", "2014-01-01", "jwst_nircam_photom_0048.fits")
assert_best_ref("SOME EXPOSURE TYPE", "2020-11-20", "jwst_nircam_photom_0048.fits")


for exposure_type in ("NRC_CORON", "NRC_FLAT", "NRC_FOCUS", "NRC_IMAGE", "NRC_TACONFIRM", "NRC_TACQ", "NRC_TSIMAGE"):
    assert_best_ref(exposure_type, "2014-01-01", "jwst_nircam_photom_0074.fits")
    assert_best_ref(exposure_type, "2020-11-21", "jwst_nircam_photom_0074.fits")
    
for exposure_type in ("NRC_CORON", "NRC_FLAT", "NRC_IMAGE", "NRC_TACONFIRM", "NRC_TACQ", "NRC_TSIMAGE"):
    assert_best_ref(exposure_type, "2020-11-20", "jwst_nircam_photom_0074.fits")

assert_best_ref("NRC_IMAGE", "2020-11-20", "jwst_nircam_photom_birthday.fits")

print("Success!")

### Challenge 2

Oh no, MIRI was struck by rogue fireworks and the previously stable flat field correction completely changed!  Add time-dependence to the following .rmap.  For each match case, use the existing reference file up to 10pm UTC on 7/4/2024, but switch to the following files (identified by band) at that time:

`LONG`: jwst_miri_flat_0600.fits

`MEDIUM`: jwst_miri_flat_0601.fits

`SHORT`: jwst_miri_flat_0602.fits

In [ ]:
RMAP_CONTENT = """
header = {
    'classes' : ('Match',),
    'derived_from' : 'jwst_miri_flat_0045.rmap',
    'filekind' : 'FLAT',
    'instrument' : 'MIRI',
    'mapping' : 'REFERENCE',
    'name' : 'jwst_miri_flat_0046.rmap',
    'observatory' : 'JWST',
    'parkey' : (('META.INSTRUMENT.DETECTOR', 'META.INSTRUMENT.FILTER', 'META.INSTRUMENT.BAND', 'META.EXPOSURE.READPATT', 'META.SUBARRAY.NAME'),),
    'sha1sum' : '1b42da81d62fb32d927911f3dcae05a980bcf939',
}

selector = Match({
    ('MIRIFULONG', 'N/A', 'LONG', 'N/A', 'FULL') : 'jwst_miri_flat_0541.fits',
    ('MIRIFULONG', 'N/A', 'MEDIUM', 'N/A', 'FULL') : 'jwst_miri_flat_0539.fits',
    ('MIRIFULONG', 'N/A', 'SHORT', 'N/A', 'FULL') : 'jwst_miri_flat_0542.fits',
})
"""

#### Test your solution

In [ ]:
rmap = crds.core.rmap.ReferenceMapping.from_string(RMAP_CONTENT, ignore_checksum=True)

def assert_best_ref(band, date, time, filename):
    result = rmap.get_best_ref(
        {
            "META.INSTRUMENT.DETECTOR": "MIRIFULONG",
            "META.INSTRUMENT.BAND": band,
            "META.SUBARRAY.NAME": "FULL",
            "META.OBSERVATION.DATE": date,
            "META.OBSERVATION.TIME": time,
        }
    )
    
    if result != filename:
        message = f"Test failed for META.EXPOSURE.BAND={band}, META.OBSERVATION.DATE={date}, META.OBSERVATION.TIME={time}.  Expected {filename}, got {result}"
        raise AssertionError(message)
        
assert_best_ref("LONG", "2023-01-01", "00:00:00", "jwst_miri_flat_0541.fits")
assert_best_ref("LONG", "2024-07-04", "09:59:59", "jwst_miri_flat_0541.fits")
assert_best_ref("LONG", "2024-07-04", "10:00:00", "jwst_miri_flat_0600.fits")
assert_best_ref("LONG", "2025-01-01", "00:00:00", "jwst_miri_flat_0600.fits")

assert_best_ref("MEDIUM", "2023-01-01", "00:00:00", "jwst_miri_flat_0539.fits")
assert_best_ref("MEDIUM", "2024-07-04", "09:59:59", "jwst_miri_flat_0539.fits")
assert_best_ref("MEDIUM", "2024-07-04", "10:00:00", "jwst_miri_flat_0601.fits")
assert_best_ref("MEDIUM", "2025-01-01", "00:00:00", "jwst_miri_flat_0601.fits")

assert_best_ref("SHORT", "2023-01-01", "00:00:00", "jwst_miri_flat_0542.fits")
assert_best_ref("SHORT", "2024-07-04", "09:59:59", "jwst_miri_flat_0542.fits")
assert_best_ref("SHORT", "2024-07-04", "10:00:00", "jwst_miri_flat_0602.fits")
assert_best_ref("SHORT", "2025-01-01", "00:00:00", "jwst_miri_flat_0602.fits")

### Challenge 3

Maybe the recipe thing?

## Further reading

The CRDS User Manual includes [detailed documentation](https://jwst-crds-bit.stsci.edu/static/users_guide/rmap_syntax.html) on the subject of CRDS rules files.